## **Librerías**

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns = False

In [3]:
from ortools.sat.python import cp_model 

## **Datos**

In [4]:
df_players = pd.read_csv('../Data/players.csv')

In [5]:
df_players.sample()

,id,name,now_cost,position,team,web_name,transfers_in_event,influence_rank_type,news_added,ict_index_rank_type,assists,selected_rank,direct_freekicks_order,starts_per_90,goals_conceded,threat_rank_type,status,ep_this,news,selected_rank_type,goals_scored,influence_rank,goals_conceded_per_90,selected_by_percent,in_dreamteam,points_per_game_rank,expected_goal_involvements_per_90,influence,form_rank_type,expected_goals_conceded_per_90,chance_of_playing_this_round,ep_next,expected_assists,expected_goals_conceded,points_per_game,cost_change_start,transfers_in,starts,cost_change_start_fall,creativity,saves_per_90,threat_rank,penalties_saved,points_per_game_rank_type,own_goals,bonus,penalties_missed,expected_goals,chance_of_playing_next_round,event_points,clean_sheets,clean_sheets_per_90,transfers_out,cost_change_event,penalties_order,expected_goals_per_90,creativity_rank_type,expected_assists_per_90,value_season,transfers_out_event,form,bps,saves,expected_goal_involvements,form_rank,creativity_rank,corners_and_indirect_freekicks_order,threat,dreamteam_count,now_cost_rank_type,ict_index,now_cost_rank,minutes,yellow_cards,cost_change_event_fall,red_cards,value_form,ict_index_rank,total_points
53,51,Tyrone Mings,41,DEF,Aston Villa,Mings,44,167,2023-08-12T21:30:06.077936Z,159,0,171,NaN,3.0,2,146,i,0.0,Knee injury - Unknown return date,64,0,461,6.0,1.2,False,605,0.51,3.6,208,3.24,0.0,0.0,0.16,1.08,0.0,-4,10034,1,4,13.0,0.0,403,0,232,0,0,0,0.01,0.0,0,0,0.0,1063230,0,NaN,0.03,137,0.48,0.0,865,0.0,5,0,0.17,518,384,NaN,5.0,0,160,2.2,626,30,0,0,0,0.0,447,0


## **Ejercicio de Optimización**

### **Restricciones**

- Tenemos que elegir 2 porteros, 5 defensas, 5 centrocampistas y 3 delanteros (un total de 15 jugadores)

- No podemos gastar más de 100 millones de libras

- No podemos elegir más de 3 jugadores del mismo equipo (por ejemplo, no podemos elegir 4 jugadores del Arsenal o 10 jugadores del Liverpool. Esta es una regla impuesta por FPL).

- Cada jugador solo puede ser elegido una vez

### **Variables Globales**

In [6]:
PRESUPUESTO = 1000

In [7]:
POSITION_MAP = {
    'Goalkeeper': {
        'code': 'GKP',
        'count': 2
    },
    'Defender': {
        'code': 'DEF',
        'count': 5
    },
    'Midfielder': {
        'code': 'MID',
        'count': 5
    },
    'Forward': {
        'code': 'FWD',
        'count': 3
    }
}

In [8]:
MAX_PLAYERS_PER_TEAM = 3

### **Primer Muestreo - Aleatorio**

In [9]:
# Elegimos a nuestro equipo de manera aleatoria
random_ids = np.random.choice(df_players['id'], 15)

random_players = df_players[df_players['id'].isin(random_ids)].copy()

In [10]:
# Calculamos el costo de nuestro equipo
team_cost = random_players['now_cost'].sum()/10
team_cost

80.5

In [11]:
# Calculamos los puntos generados por nuestro equipo
team_points = random_players['points_per_game'].sum()
team_points

19.6

### **Modelo - Definición de las restricciones**

In [12]:
model = cp_model.CpModel()

#### **Definimos a nuestros jugadores**

In [13]:
# Inicializamos un diccionario en el que almacenaremos los jugadores con sus ids
decision_variables = {}

for position, details in POSITION_MAP.items():
    players = list(df_players['id'])

    player_variables = {i: model.NewBoolVar(f'player_{i}') for i in players}
    decision_variables.update(player_variables)

#### **Restricción 1 - Número de jugadores por posición**

In [13]:
# Vamos a agregar la restricción para cada posición
decision_variables = {}


for position, details in POSITION_MAP.items():
    
    # Definimos los jugadores para cada posición y la cantidad de jugadores necesarios para nuestro equipo
    players_in_position = list(df_players.query(f'position == "{details['code']}"')['id']) 
    player_count = details['count']

    # Creamos un diccionario binario para los jugadores
    player_variables = {i: model.NewBoolVar(f'player_{i}') for i in players_in_position}

    # player_variables = {i: model.NewBoolVar(f'player_{i}') for i in players}
    decision_variables.update(player_variables)

    # Sumamos el total de jugadores seleccionados
    restriction_player_positions = sum(player_variables.values())

    # Agregamos la restricción a nuestro modelo    
    model.Add(restriction_player_positions == player_count)

#### **Restricción 2 - Presupuesto**

In [14]:
# Creamos un diccionario que contiene el costo de cada jugador
player_costs = {
    player: df_players[df_players['id']==player]['now_cost'].values[0] for player in df_players['id']
}

# Calculamos el costo toal de los jugadores
restriction_cost = {var * player_costs[i] for i, var in decision_variables.items()}

# Agregamos la restricción de presupuesto a nuestro modelo
model.add(sum(restriction_cost) <= PRESUPUESTO)

#### **Restricción 3 - Jugadores del mismo equipo**

In [15]:
teams = df_players['team'].unique()

In [16]:
# Agregamos la restricción para cada equipo
for team in teams:
    # Seleccionamos los jugadores del equipo en cuestión
    eligible_players = df_players[df_players['team']==team]['id'].values
    
    # Calculamos la cantidad de jugadores disponibles
    restriction_players_team = sum(decision_variables[i] for i in eligible_players)

    # Agregamos la restricción al modelo
    model.Add(restriction_players_team <= MAX_PLAYERS_PER_TEAM)

#### **Función Objetivo**

In [17]:
# Tomaremos como variable objetivo la cantidad de puntos promedio por juego
df_players[['name', 'team','now_cost', 'position', 'points_per_game']].sort_values('points_per_game', ascending=False).head(10)

,name,team,now_cost,position,points_per_game
400,Mohamed Salah,Liverpool,133,MID,7.6
474,Erling Haaland,Man City,139,FWD,7.5
313,André Tavares Gomes,Everton,44,MID,7.0
254,Christopher Nkunku,Chelsea,73,FWD,7.0
688,Son Heung-min,Spurs,98,MID,6.8
61,Ollie Watkins,Aston Villa,88,FWD,6.1
706,Jarrod Bowen,West Ham,78,MID,6.0
316,Seamus Coleman,Everton,44,DEF,6.0
17,Bukayo Saka,Arsenal,90,MID,5.8
125,Bryan Mbeumo,Brentford,68,MID,5.8


In [18]:
# Generamos un dicccionario con los puntos por jugador
player_points = {
    player: df_players[df_players['id']==player]['points_per_game'].values[0] 
    for player in decision_variables.keys()
}

In [19]:
total_points = sum((var * player_points[i] for i, var in decision_variables.items()))

In [20]:
model.Maximize(total_points)

#### **Solución**

In [21]:
solver = cp_model.CpSolver()

In [22]:
status = solver.Solve(model)

In [23]:
if status == cp_model.OPTIMAL:
    print('Se encuentra la solución óptima')

Se encuentra la solución óptima


In [30]:
fantasy_team = []

for i, var in decision_variables.items():
    if solver.Value(var) == 1:
        player_id = df_players.query(f'id == {i}')['id'].values[0]
        
        fantasy_team.append(player_id)

In [37]:
df_fantasy = df_players[df_players['id'].isin(fantasy_team)].copy()

In [40]:
df_fantasy[['name', 'team', 'position', 'now_cost', 'points_per_game']]

,name,team,position,now_cost,points_per_game
61,Ollie Watkins,Aston Villa,FWD,88,6.1
125,Bryan Mbeumo,Brentford,MID,68,5.8
163,Solly March,Brighton,MID,61,5.6
254,Christopher Nkunku,Chelsea,FWD,73,7.0
313,André Tavares Gomes,Everton,MID,44,7.0
316,Seamus Coleman,Everton,DEF,44,6.0
327,Vitalii Mykolenko,Everton,DEF,46,4.8
382,Trent Alexander-Arnold,Liverpool,DEF,83,5.6
403,Konstantinos Tsimikas,Liverpool,DEF,48,4.5
474,Erling Haaland,Man City,FWD,139,7.5
